In [ ]:
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
import pandas as pd

COLUMNS = ['product_name', 'product_price', 'product_rating', 'product_review_count', 'product_url']

def page_url(page_number):
    url = f'https://www.amazon.com/s?i=computers-intl-ship&bbn=16225007011&rh=n%3A16225007011%2Cn%3A172456&page={page_number}'
    return url

def get_soup(url):
    driver = uc.Chrome()
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    driver.quit()
    return soup

def get_product_infos(soup):
    products = []
    all_products_infos = soup.select('div.a-section.a-spacing-small.puis-padding-left-small.puis-padding-right-small')
    for product in all_products_infos:
        product_name = product.select('span.a-size-base-plus.a-color-base.a-text-normal')[0].text if product.select('span.a-size-base-plus.a-color-base.a-text-normal') else None
        product_price = product.select('span.a-offscreen')[0].text if product.select('span.a-price-whole') else None
        product_rating = product.select('span.a-icon-alt')[0].text.split(' ')[0] if product.select('span.a-icon-alt') else None
        product_review_count = product.select('span.a-size-base')[0].text if product.select('span.a-size-base') else None
        product_url = 'https://www.amazon.com' + product.select('a.a-link-normal.a-text-normal')[0]['href'] if product.select('a.a-link-normal.a-text-normal') else None 
        product_info = [product_name, product_price, product_rating, product_review_count, product_url]
        products.append(product_info)
    return products

df = pd.DataFrame(columns=COLUMNS)
page_number = 1
soup_old = None
soup_new = get_soup(url=page_url(page_number=page_number))
while soup_old != soup_new:
    products = get_product_infos(soup=soup_new)
    df = pd.concat([df, pd.DataFrame(products, columns=COLUMNS)], ignore_index=True)
    soup_old = soup_new
    page_number += 1
    soup_new = get_soup(url=page_url(page_number))
    
df = df.drop_duplicates(subset=COLUMNS).reset_index(drop=True)

In [42]:
df

,product_name,product_price,product_rating,product_review_count,product_url
0,"Cã-vǐ-ta-Tion Ma-Chine, Beaǖty Ma-Chine, Skǐn ...",$299.00,5.0,3,https://www.amazon.com/sspa/click?ie=UTF8&spc=...
1,"VEVOR 6U Wall Mount Network Server Cabinet, 15...",$118.36,3.1,5,https://www.amazon.com/sspa/click?ie=UTF8&spc=...
2,"Onten 10Gbps USB C Hub,USB C to USB C/A Adapte...",$22.99,4.8,10,https://www.amazon.com/sspa/click?ie=UTF8&spc=...
3,SAMSUNG EVO Select Micro SD-Memory-Card + Adap...,$17.99,4.7,"55,842",https://www.amazon.com/SAMSUNG-microSDXC-Expan...
4,Logitech MK270 Wireless Keyboard And Mouse Com...,None,4.5,"84,188",https://www.amazon.com/Logitech-MK270-Wireless...
...,...,...,...,...,...
442,"Computer Speakers, Bluetooth Soundbar, HiFi St...",$42.49,4.3,"8,453",https://www.amazon.com/Computer-Speakers-Dynam...
443,[10Gbps] USB C Female to USB Male Adapter (2-P...,$8.99,4.5,"7,660",https://www.amazon.com/Adapter-XAOSUN-One-Side...
444,Brother Printer LC3011BK Singe Pack Standard C...,$13.98,4.6,"6,477",https://www.amazon.com/Brother-Printer-LC3011B...
445,Mouse Jiggler Undetectable Metal USB Mouse Mov...,$13.99,4.7,"2,571",https://www.amazon.com/Undetectable-Automatic-...
